In [9]:
%pwd

'/workspaces/Text-Summarization'

Incase the pwd is not the home folder, change the directory to home folder
import os
os.chdir("../")

In [2]:
import os
os.chdir("../")

In [10]:
%pwd

'/workspaces/Text-Summarization'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [12]:
from src.textSummarization.constants import *
from src.textSummarization.utils.common import read_yaml, create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            ):
        print("Configuration Manager Initiated")
        # print(f"Config File Path: {config_filepath}, with dtype {type(config_filepath)}")
        # print(f"Params File Path: {params_filepath} with dtype {type(params_filepath)}")
        self.configs = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # print(f"Configs: {self.configs}")
        create_directories([self.configs.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.configs.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfig(
            root_dir = Path(config.root_dir),
            source_url = config.source_url,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

# config_dummy = ConfigurationManager()


In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
from src.textSummarization.logging import logger
from src.textSummarization.utils.common import get_size
from src.textSummarization.entity.config_entity import DataIngestionConfig

In [14]:
class DataIngestion:
    """
    Data Ingestion class
    """
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        """
        Download file from source url
        """
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info("Downloaded file %s with the folowing headers: \n%s", filename, headers)
        else:
            logger.info(
                "File already exists at %s of size %s",
                  self.config.local_data_file,
                    get_size(self.config.local_data_file))

    def extract_zip_file(self):
        """
        extract the zip file
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with ZipFile(self.config.local_data_file, 'r') as zip_file:
            zip_file.extractall(path=unzip_path)
            logger.info("Extracted file at %s", self.config.unzip_dir)

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    logger.exception(e)
    raise e
    

Configuration Manager Initiated
[2023-10-04 18:16:54,373: INFO: common] Loaded yaml file: config/config.yaml
[2023-10-04 18:16:54,375: INFO: common] Loaded yaml file: params.yaml
[2023-10-04 18:16:54,376: INFO: common] Created directory: artifacts
[2023-10-04 18:16:54,376: INFO: common] Created directory: artifacts/data_ingestion
[2023-10-04 18:16:54,375: INFO: common] Loaded yaml file: params.yaml
[2023-10-04 18:16:54,376: INFO: common] Created directory: artifacts
[2023-10-04 18:16:54,376: INFO: common] Created directory: artifacts/data_ingestion
[2023-10-04 18:16:55,353: INFO: 3469703988] Downloaded file artifacts/data_ingestion/data.zip with the folowing headers: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nos